In [1]:
import pandas as pd
import numpy as np
import re
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('properatti.csv')
title = data['title']
description = data['description']

In [3]:
data['price_aprox_usd'] = data['price_aprox_usd'].round()
data['price_aprox_usd'] = data['price_aprox_usd'].astype('int', errors='ignore')
pd.options.display.max_columns = None

In [4]:
#buscamos mediante un patron y el uso de expresiones regulares los precios en USD que estan en la columna de title
patron = '^U\$D \d*.\d*.\d*'
title_regex = re.compile(patron)
resultado = title.apply(lambda x:  x if x is np.NaN else title_regex.search(x))

#buscamos mediante un patron y el uso de expresiones regulares los precios en USD que estan en la columna de description
patron_d = 'U\$S \d*.\d*.\d*'
description_regex = re.compile(patron_d)
resultado_d = description.apply(lambda x:  x if x is np.NaN else description_regex.search(x))

#buscamos mediante un patron y el uso de expresiones regulares los m2 que estan en la columna de title
patron_m = '(?:\d+ (?:m2|M2)|\d+(?:m2|M2))'
title_regex_m = re.compile(patron_m)
resultado_m = title.apply(lambda x:  x if x is np.NaN else title_regex_m.search(x))

In [5]:
# las coincidencias con el patron se volcaron a nueva columna Titleclean
mask_notnull = resultado.notnull()
data.loc[mask_notnull, "TitleClean"] = resultado[mask_notnull].apply(lambda x: x.group(0))
dolar= data.loc[mask_notnull, ["TitleClean"]]

# las coincidencias con el patron se volcaron a nueva columna Descriptionclean
mask_notnull_d = resultado_d.notnull()
data.loc[mask_notnull_d, "DescriptionClean"] = resultado_d[mask_notnull_d].apply(lambda x: x.group(0))
dolar_d= data.loc[mask_notnull_d, ["DescriptionClean"]]

# las coincidencias con el patron se volcaron a nueva columna Title_m2
mask_notnull_m = resultado_m.notnull()
data.loc[mask_notnull_m, "Title_m2"] = resultado_m[mask_notnull_m].apply(lambda x: x.group(0))
dolar_m= data.loc[mask_notnull_m, ["Title_m2"]]

In [6]:
# Cambiamos el type a string, ya que las funciones que continuan emitian error si no era tipo string.
title_clean = data['TitleClean']
title_clean2 = title_clean.apply(str)
description_clean = data['DescriptionClean']
description_clean2 = description_clean.apply(str)
title_clean_m = data['Title_m2']
title_clean_m2 = title_clean_m.apply(str)

In [7]:
# mediante el uso de un nuevo patron quitamos el signo U$D para quedarnos solo con el precio (sin la moneda)
patron_usd = '\d+\d+\d+'
title_clean2_regex = re.compile(patron_usd)
resultado2 = title_clean2.apply(lambda x:  x if x is np.NaN else title_clean2_regex.search(x))

# mediante el uso de un nuevo patron quitamos el signo U$S para quedarnos solo con el precio (sin la moneda)
patron_usd_d = '\d+\d+\d+'
description_clean2_regex = re.compile(patron_usd_d)
resultado2_d = description_clean2.apply(lambda x:  x if x is np.NaN else description_clean2_regex.search(x))

# mediante el uso de un nuevo patron quitamos el signo m2 para quedarnos solo con la superficie
patron_m2 = '^\d+'
title_clean_m2_regex = re.compile(patron_m2)
resultado2_m = title_clean_m2.apply(lambda x:  x if x is np.NaN else title_clean_m2_regex.search(x))

In [8]:
# lo volcamos a una nueva columna SinUSD
mask2_notnull = resultado2.notnull()
data.loc[mask2_notnull, "SinUSD"] = resultado2[mask2_notnull].apply(lambda x: x.group(0))
dolar2= data.loc[mask2_notnull, ["SinUSD"]]

In [9]:
# Eliminamos los puntos y lo convertimos en float
data["SinUSD"] = data["SinUSD"].replace(to_replace =".", value ="").astype(int, errors='ignore')
#data[data["SinUSD"].notnull()]

In [10]:
# lo volcamos a una nueva columna SinUSS
mask2_notnull_d = resultado2_d.notnull()
data.loc[mask2_notnull_d, "SinUSS"] = resultado2_d[mask2_notnull_d].apply(lambda x: x.group(0))
dolar2_d= data.loc[mask2_notnull_d, ["SinUSS"]]

# lo volcamos a una nueva columna Sin_m2
mask2_notnull_m = resultado2_m.notnull()
data.loc[mask2_notnull_m, "Sin_m2"] = resultado2_m[mask2_notnull_m].apply(lambda x: x.group(0))
dolar2_m= data.loc[mask2_notnull_m, ["Sin_m2"]]

In [11]:
#completamos los campos vacios de precios en usd por lo nuevos ,sacados de title, reflejados en SinUSD
data.price_aprox_usd.fillna(data.SinUSD, inplace=True)

#completamos los campos vacios de precios en usd por lo nuevos ,sacados de description, reflejados en SinUSS
data.price_aprox_usd.fillna(data.SinUSS, inplace=True)

#completamos los campos vacios de metros en m2 por lo nuevos ,sacados de title, reflejados en Sin_m2
data.surface_total_in_m2.fillna(data.Sin_m2, inplace=True)

In [12]:
#creamos un sub data solo con los valores de Capital Federal, para reducir el dataset y enfocar el análisis en una de las zonas con mayor cantidad de registros.
caba = data.state_name == "Capital Federal"
data_caba = data [caba]

In [13]:
# Dropeamos las filas que tienen el precio nulo dado que no era una cantidad significativa de casos y no se podía conseguir con información de otras columnas del dataset
data_caba.dropna(subset=['price_aprox_usd'], inplace=True)

# Quitamos el warning resultante con la siguiente función:
pd.options.mode.chained_assignment = None

In [14]:
#Convertimos a flotante los datos de la columna superficie total en m2
data_caba['surface_total_in_m2'] = data_caba['surface_total_in_m2'].apply(lambda x: float(x))

In [15]:
#Convertimos a flotante los datos de la columna precio en dolares
data_caba['price_aprox_usd'] = data_caba['price_aprox_usd'].apply(lambda x: float(x))

In [48]:
# Analizamos los outliers en la columna precio aprox en dolar
data_caba['id'] = np.arange(len(data_caba))

In [50]:
# Graficamos los precios por barrio

app = Dash(__name__)
data= data_caba

app.layout = html.Div([
    html.H4('Interactive scatter plot with Iris dataset'),
    dcc.Graph(id="scatter-plot"),
    html.P("Filter by petal width:"),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=10000000, step=1000,
        marks={0: '0', 10000000: '10.000.000'},
        value=[10000,450000 ]
    ),
])


@app.callback(
    Output("scatter-plot", "figure"), 
    Input("range-slider", "value"))
def update_bar_chart(slider_range):
    df = data # replace with your own data source
    low, high = slider_range
    mask = (data['price_aprox_usd'] > low) & (data['price_aprox_usd'] < high)
    fig = px.scatter(
        data[mask], x="place_name", y="price_aprox_usd", 
        color="price_aprox_usd", size='price_aprox_usd', 
        hover_data=['id'])
    return fig


app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [36]:
# En base al grafico se identifican los outliers y se quitan de la base
outlayers = (data_caba['price_aprox_usd'] > 20000000) | (data_caba['price_aprox_usd'] < 15000)
outlayers_index = data_caba[outlayers].index
data_caba = data_caba.drop(outlayers_index)

In [39]:
#Agrupamos por barrio
dataset_groupby_barrio = data_caba.groupby('place_name')

In [40]:
#Completamos los valores nulos de price_usd_per_m2 (superficie total) con el promedio segun su barrio, usando transform y lambda.
price_m2_usd_filled = dataset_groupby_barrio['price_usd_per_m2'].transform(lambda grp: grp.fillna(grp.mean()))

In [41]:
#Completamos los valores nulos de price_per_m2 (superficie cubierta) con el promedio segun su barrio, usando transform y lambda.
price_m2_filled = dataset_groupby_barrio['price_per_m2'].transform(lambda grp: grp.fillna(grp.mean()))

In [42]:
#Agregamos la columna de m2_usd_filled al dataset
data_caba.loc[:, ['price_usd_m2_filled']] = price_m2_usd_filled
price_usd_fill = data_caba['price_usd_m2_filled']

In [43]:
#Agregamos la columna de m2_usd_filled al dataset
data_caba.loc[:, ['price_m2_filled']] = price_m2_filled
price_m2_fill = data_caba['price_m2_filled']

In [44]:
# Para controlar imprimimos la cantidad de nulos en precio por m2
print(data_caba.price_usd_m2_filled	.isnull().sum())
print(data_caba.price_m2_filled.isnull().sum())

0
0


In [51]:
# Completamos los mts2 vacios con la informacion de price_usd_per_m2 (superficie total) / price_aprox_usd a completrar en la columna surface_total_in_m2 

print(data_caba.surface_total_in_m2.isnull().sum())

5391


In [45]:
# Creamos rangos de m2 con un pd.cut para falicitar el análisis de los tipos de propiedades
bins = [0,50,100,300000]
cuantil = pd.cut(data_caba['surface_total_in_m2'], bins, right=False)
cuantil.value_counts()

[50, 100)        8454
[0, 50)          8293
[100, 300000)    6846
Name: surface_total_in_m2, dtype: int64

In [46]:
pd.options.display.max_rows = None
grp_barrio = data_caba.groupby(['property_type',cuantil])[['price_usd_m2_filled','price_aprox_usd']].mean().round(2)
grp_barrio

price_usd_m2_filled  price_aprox_usd
property_type surface_total_in_m2                                      
PH            [0, 50)                          2456.84         86892.78
              [50, 100)                        1969.10        135293.64
              [100, 300000)                    1441.22        244715.86
apartment     [0, 50)                          4039.99        146417.36
              [50, 100)                        2650.55        180853.53
              [100, 300000)                    3207.48        584972.42
house         [0, 50)                          7796.38        273672.00
              [50, 100)                        2547.57        186173.21
              [100, 300000)                    1403.73        546108.29
store         [0, 50)                          3662.74        116357.10
              [50, 100)                        3482.97        243876.15
              [100, 300000)                    2483.28        794791.06

In [47]:
# Mostramos el dataset conformado con un ejemplo de 10 filas
data_caba.head(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,TitleClean,DescriptionClean,Title_m2,SinUSD,SinUSS,Sin_m2,price_usd_m2_filled,price_m2_filled
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,NaN,NaN,NaN,NaN,NaN,NaN,1127.272727,1550.000000
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,NaN,NaN,NaN,NaN,NaN,NaN,1309.090909,1309.090909
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,NaN,NaN,NaN,NaN,NaN,NaN,1803.908335,2278.839694
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,NaN,NaN,NaN,NaN,NaN,NaN,3066.666667,3450.000000
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,NaN,NaN,NaN,NaN,NaN,NaN,3000.000000,3250.000000
13,13,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0,NaN,http://www.properati.com.ar/15bp9_venta_depart...,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,https://thumbs4.properati.com/2/ZTUjkRzTz6YvcU...,NaN,NaN,NaN,NaN,NaN,NaN,2234.000000,3723.333333
14,14,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0,NaN,http://www.properati.com.ar/15bpa_venta_depart...,Torre II Dalí. Ambiente unico divisible.Vitrau...,Vitraux Palermo,https://thumbs4.properati.com/8/Uay2dC732CNlRw...,NaN,NaN,NaN,NaN,NaN,NaN,3521.428571,4770.967742
16,16,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6523561177,-58.5016239381",-34.652356,-58.501624,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0,NaN,http://www.properati.com.ar/15bpj_venta_ph_mat...,Ventas Mataderos al frente Duplex 4 amb.- Plan...,VENTA-MATADEROS-DUPLEX 4 AMB.,https://thumbs4.properati.com/7/XSdJtiE8ZkWgon...,NaN,NaN,NaN,NaN,NaN,NaN,1707.14